In [4]:
import pandas as pd
import string
import numpy as np
import sklearn.metrics as metrics
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from nltk.tokenize import word_tokenize
import nltk
import gensim
from gensim.models import Word2Vec
from sklearn.ensemble import AdaBoostClassifier

In [3]:
pip install -U gensim

  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Stored in directory: C:\Users\bhanu\AppData\Local\pip\Cache\wheels\ab\10\93\5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df = pd.read_csv('C:/AIT 590/Sree-Final Project/yelp_review.csv/yelp_review.csv')

In [3]:
df1 = df.sample(30000)

In [4]:
stop = stopwords.words('english')

In [5]:
df1['stopwords'] = df1['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
# df1[['text','stopwords']].head()
df1['numerics'] = df1['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
# df1[['text','numerics']].head()

In [6]:
df1['text length'] = df1['text'].apply(len)
df1.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,stopwords,numerics,text length
914207,6zbLZbN_MqxcrlWRgrM2yA,rmmKsvA4WOvUD_kFrjxPoQ,jiOREht1_iH8BPDBe9kerw,4,2010-11-04,I've gone here twice now and I can say this pl...,0,0,0,20,0,285
4831231,YOyCEKvBRcpNApqDVWlXYg,lUncKI0Zf9pQn4-nFeLuPQ,VRwT0pscR5vESCrAnUpNwQ,5,2016-01-17,Amazing good food! Nothing that hasn't been sa...,1,0,1,40,2,534
1487495,bejUYIZUTCFD4RrMfLL7Jw,9hMqNBBpTl3lBy6qCAyu-A,BTcY04QFiS1uh-RpkR7rAg,5,2011-05-19,"Childcare, distance from my home, away from hu...",1,0,1,16,0,212
4815894,nzu2Q-aMWIRf7IH1LVyzsA,JUl-g1ZpfAIHXeucnns84A,k1QpHAkzKTrFYfk6u--VgQ,4,2013-09-02,yay! great atmosphere. great decor! pizza was ...,1,1,1,3,0,88
1359260,J7gZsqBqtKXodh5ht86u9g,jW37YOkSmBUc4NSf9WF4Ng,o_vXKqJ6kPkT9RTSNNRLCQ,5,2015-01-03,My regular place and Manny is the man. \n\nIt ...,0,0,0,15,0,266


In [7]:
df2 = df1[['text','stars']].copy()
df2.head()

,text,stars
914207,I've gone here twice now and I can say this pl...,4
4831231,Amazing good food! Nothing that hasn't been sa...,5
1487495,"Childcare, distance from my home, away from hu...",5
4815894,yay! great atmosphere. great decor! pizza was ...,4
1359260,My regular place and Manny is the man. \n\nIt ...,5


In [8]:
df2['text'] = df2['text'].str.lower()

In [9]:
%%time
df2['token_text'] = df2.text.apply(lambda x: word_tokenize(x))

CPU times: user 33.2 s, sys: 312 ms, total: 33.6 s
Wall time: 33.7 s


In [10]:
def clean(text):
    cleaned = [w for w in text if w not in stop]
    cleaned = [w for w in cleaned if w not in string.punctuation]
    return ' '.join(cleaned)

In [11]:
%%time
df2['clean_text'] = df2['token_text'].apply(clean)
df2['text'] = df2['clean_text']
df2.drop(['clean_text','token_text'],1,inplace=True)

CPU times: user 7.14 s, sys: 34 ms, total: 7.18 s
Wall time: 7.18 s


In [12]:
df2.head()

,text,stars
914207,'ve gone twice say place pretty awesome quick ...,4
4831231,amazing good food nothing n't said experienced...,5
1487495,childcare distance home away hustle malls plac...,5
4815894,yay great atmosphere great decor pizza delicio...,4
1359260,regular place manny man barbershop better supe...,5


In [13]:
%%time
cv = TfidfVectorizer(min_df=5)
X = cv.fit_transform(df2.text)

CPU times: user 1.77 s, sys: 40.1 ms, total: 1.81 s
Wall time: 1.81 s


In [14]:
%%time
features = pd.DataFrame(X.toarray(),columns=cv.get_feature_names())

CPU times: user 837 ms, sys: 1.31 s, total: 2.15 s
Wall time: 4.04 s


In [15]:
%%time
xtr1,xts1,ytr1,yts1 = train_test_split(features,df2.stars)
lr = LogisticRegression()
lr.fit(xtr1,ytr1)

CPU times: user 4min 25s, sys: 6.78 s, total: 4min 32s
Wall time: 2min 12s


/Users/anshulgoyal/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [16]:
preds_lr1 = lr.predict(xts1)

In [17]:
accuracy_score(yts1,preds_lr1)

0.6293333333333333

In [ ]:
wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
%%time
# 30000 sample size
docs_vectors = pd.DataFrame()

for doc in df2['text'].str.replace('\d+', ''):
    temp = pd.DataFrame()   
    for word in doc.split(' '): 
        if word not in stopwords: 
            try:
                word_vec = wv[word]  
                temp = temp.append(pd.Series(word_vec), ignore_index = True)  
            except:
                pass
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) 
docs_vectors.shape

In [ ]:
docs_vectors.shape

In [ ]:
docs_vectors['stars'] = list(df2.stars)

In [ ]:
docs_vectors = docs_vectors.dropna()

In [ ]:
pd.isnull(docs_vectors).sum().sum()

In [ ]:
vectors = docs_vectors.drop(['stars'],1)
stars = docs_vectors['stars']

In [ ]:
xtr,xts,ytr,yts = train_test_split(vectors,stars)

In [ ]:
%%time
lr = LogisticRegression()
lr.fit(xtr,ytr)

In [ ]:
preds_lr = lr.predict(xts)

In [ ]:
accuracy_score(yts,preds_lr)

In [ ]:
%%time
ad = AdaBoostClassifier()
ad.fit(xtr,ytr)

In [ ]:
preds_ad = ad.predict(xts)

In [ ]:
accuracy_score(yts,preds_ad)